In [1]:
import json

In [6]:
keys = ['train', 'dev', 'test']
processed_data = {k:[] for k in keys}
lengths ={k:[] for k in keys}
for k in keys :
    data = [json.loads(line) for line in open('complete-movies-data/' + k + '.jsonl')]
    for d in data :
        doc_id = d['annotation_id']
        text = open('complete-movies-data/docs/' + doc_id).read()
        lengths[k].append(sum([(e['end_token'] - e['start_token']) for eg in d['evidences'] for e in eg]) / len(text.split()))
        label = d['classification']
        processed_data[k].append({'document' : text, 'label' : d['classification'], 'annotation_id' : d['annotation_id']})

In [8]:
for k in keys :
    f = open('data/' + k + '.jsonl', 'w')
    f.write('\n'.join([json.dumps(line) for line in processed_data[k]]))
    f.close()

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
tfidf = TfidfVectorizer()
X_train = tfidf.fit_transform([d['document'] for d in processed_data['train']])
X_dev = tfidf.transform([d['document'] for d in processed_data['dev']])

y_train = [int(d['label'] == 'POS') for d in processed_data['train']]
y_dev = [int(d['label'] == 'POS') for d in processed_data['dev']]

In [39]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2')
lr.fit(X_train, y_train)
lr.score(X_dev, y_dev)

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8